In [102]:
from selenium import webdriver
import time
import os
from gsheets import Sheets
import pandas as pd
import operator
import mysql.connector
import shutil
from shutil import copyfile
import os.path
import csv
from gspread_pandas import Spread, Client 
from oauth2client.service_account import ServiceAccountCredentials
import pprint
import gspread
import math


In [103]:
#reading credientials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

#wd for authentiction file
os.chdir('/Users/innovatus6')
creds = ServiceAccountCredentials.from_json_keyfile_name('linkupdater-631ef6e77556.json', scope)


In [104]:
#getting permission 
client = gspread.authorize(creds)

In [105]:
#opening production sheet
requests= client.open("Master Data Builds").worksheet("Priority Builds -requests")
primary= client.open("Master Data Builds").worksheet("Priority Builds")

In [106]:
df = pd.DataFrame(requests.get_all_records())

In [107]:
harvestfield=df.loc[operator.and_(df.type == 'notfounder', df.type != 'used')]

In [108]:
harvestfield=harvestfield.reset_index(drop=True)
harvestfield

,event,founderid,type,link
0,cf_emea_623_,"it function, from founder 5",notfounder,https://www.linkedin.com/sales/search/people?c...
1,cf_emea_623_,"it function, from founder 6",notfounder,https://www.linkedin.com/sales/search/people?c...
2,cf_emea_623_,"it function, from founder 7",notfounder,https://www.linkedin.com/sales/search/people?c...
3,cf_emea_623_,"it function, from founder 8",notfounder,https://www.linkedin.com/sales/search/people?c...
4,cf_emea_623_,"it function, from founder 9",notfounder,https://www.linkedin.com/sales/search/people?c...
5,cf_emea_623_,"it function, from founder 10",notfounder,https://www.linkedin.com/sales/search/people?c...
6,cf_emea_623_,"it function, from founder 11",notfounder,https://www.linkedin.com/sales/search/people?c...
7,cf_emea_623_,"it function, from founder 12",notfounder,https://www.linkedin.com/sales/search/people?c...
8,cf_emea_623_,"it function, from founder 13",notfounder,https://www.linkedin.com/sales/search/people?c...
9,cf_emea_623_,"it function, from founder 14",notfounder,https://www.linkedin.com/sales/search/people?c...


In [109]:
##the below code fills in the registration page and submits it

In [110]:
#toggleing linkedin login page
web = webdriver.Chrome('/Users/innovatus6/chromedriver')
time.sleep(5)
web.get("https://www.linkedin.com/home")

In [111]:
#pressing linkedin login button
button=web.find_element_by_xpath('/html/body/nav/div/a[2]')
time.sleep(5)
web.execute_script("arguments[0].click();", button)

In [112]:
#email
email="noah@innovatusmedia.com.au"
emailslot=web.find_element_by_xpath('//*[@id="username"]')
time.sleep(1)
emailslot.send_keys(email)


#pw
pw="Innov8us"
firstslot=web.find_element_by_xpath('//*[@id="password"]')
time.sleep(1)
firstslot.send_keys(pw)


#submitting
buttonsub=web.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
time.sleep(5)
web.execute_script("arguments[0].click();", buttonsub)
time.sleep(15)

In [113]:
#skipping
buttonskip=web.find_element_by_xpath('//*[@id="ember15"]/button[2]')
time.sleep(5)
buttonskip.click()


In [114]:
#toggling sales nav page
time.sleep(5)
web.get("https://www.linkedin.com/uas/login?session_redirect=/sales&fromSignIn=true&trk=navigator")

In [115]:
#submitting pw again

pw="Innov8us"
firstslot=web.find_element_by_xpath('//*[@id="password"]')
time.sleep(1)
firstslot.send_keys(pw)
time.sleep(1)


In [116]:
#clicking sign in button
buttonsub=web.find_element_by_xpath('//*[@id="app__container"]/main/div[2]/form/div[3]/button')
time.sleep(5)
web.execute_script("arguments[0].click();", buttonsub)
time.sleep(5)

In [117]:
#definitions: 
#founder link = geo + companies
#source link = geo + companies + title/function

#toggling build link

for i in range(0,len(harvestfield.link)):
    
    link=web.get(harvestfield.link[i])
    linktext=harvestfield.link[i]
    
    time.sleep(5)
    
    
    #static first part of url
    start='https://www.linkedin.com/sales/search/people?companyIncluded='

    #last part of url
    finish=linktext[linktext.find('&companyTimeScope=CURRENT'):]

    #company parses
    companyparse=linktext[linktext.find("?companyIncluded=")+17:linktext.find('&companyTimeScope=CURRENT')]

    #isolating the part of the url corresponding to the company url
    companyparselist=companyparse.split('%2C')
    
    
    
    
####### use while loop and to reduce the parselist as means to split links
#######    

    #j controls how large of a portion out of companyparse do we sample
    j=2

    while len(companyparselist) != 0:
        
        
        #we start from the bottom and work on ourway up
        segmentedlist=companyparselist[math.floor(len(companyparselist)*(j-1)/j):]
        #compiling said company parses
        slcompiled='%2C'.join(segmentedlist)
        #creating the final link 
        segmentedlink=start + slcompiled + finish
        
        #getting new linkcount 
        web.get(segmentedlink)
        time.sleep(10)
        
        link_count = web.find_element_by_xpath('//*[@id="search-spotlight-tab-ALL"]/span[1]')
        
        
        if link_count.text.isdigit():
                
            #defining the row to append
            row=['',harvestfield.event[i],harvestfield.founderid[i],'','','',int(link_count.text),segmentedlink]

            #appending to the primary sheet
            primary.append_row(row,table_range='A1')

            #trimming the list so that the next iteration we shall work with a fresh, smaller pool of companies
            companyparselist = [company for company in companyparselist if company not in segmentedlist]
            
            print("segmentedlist is now "+ slcompiled)
            print("companyparselist is now "+ '%2C'.join(companyparselist))

        else:
            
            #else...we shall select the a smaller section of the list towards the end
            j=j+4
            print("j is now: "+ str(j))

            
        
        
        print(link_count.text)     
    
    #updating the requests sheet when a source link is done
    requests.update_cell(int(df[df.iloc[:,3]==harvestfield.link[i]].index.to_numpy())+2,3,"used")

    
    
    
web.quit()
        
        


j is now: 6
3.5K+
j is now: 10
1K+
segmentedlist is now Liverpool%2520City%2520Council%3A22535%2CRoyal%2520Society%2520of%2520Chemistry%3A23105%2CAdresseavisen%3A23178%2CNorthamptonshire%2520County%2520Council%3A23313%2CPaddy%2520Power%2520Betfair%3A23324
companyparselist is now Smith%2520%2526%2520Williamson%3A18322%2CStaffordshire%2520University%3A18398%2CUniversity%2520of%2520South%2520Wales%3A18566%2CTK%2520Maxx%3A18676%2CMenzies%2520Aviation%3A18698%2CVeikkaus%3A18983%2CMet%2520Office%3A19195%2CITPS%3A19279%2CUniversity%2520of%2520Wolverhampton%3A19289%2CUniversity%2520of%2520Sunderland%3A19398%2CUniversity%2520of%2520West%2520London%3A19580%2CSouthern%2520Water%3A19581%2CManchester%2520City%2520Council%3A19687%2CPortsmouth%2520City%2520Council%3A19876%2CGrant%2520Thornton%2520Sweden%3A20119%2CDevon%2520County%2520Council%3A20170%2COxfordshire%2520County%2520Council%3A20203%2CMacfarlanes%3A20286%2CGlasgow%2520City%2520Council%3A20348%2CSuffolk%2520County%2520Council%3A20394%2CLond

segmentedlist is now Portsmouth%2520City%2520Council%3A19876%2CGrant%2520Thornton%2520Sweden%3A20119
companyparselist is now Smith%2520%2526%2520Williamson%3A18322%2CStaffordshire%2520University%3A18398%2CUniversity%2520of%2520South%2520Wales%3A18566%2CTK%2520Maxx%3A18676%2CMenzies%2520Aviation%3A18698%2CVeikkaus%3A18983%2CMet%2520Office%3A19195%2CITPS%3A19279%2CUniversity%2520of%2520Wolverhampton%3A19289%2CUniversity%2520of%2520Sunderland%3A19398%2CUniversity%2520of%2520West%2520London%3A19580%2CSouthern%2520Water%3A19581%2CManchester%2520City%2520Council%3A19687
257
segmentedlist is now Southern%2520Water%3A19581%2CManchester%2520City%2520Council%3A19687
companyparselist is now Smith%2520%2526%2520Williamson%3A18322%2CStaffordshire%2520University%3A18398%2CUniversity%2520of%2520South%2520Wales%3A18566%2CTK%2520Maxx%3A18676%2CMenzies%2520Aviation%3A18698%2CVeikkaus%3A18983%2CMet%2520Office%3A19195%2CITPS%3A19279%2CUniversity%2520of%2520Wolverhampton%3A19289%2CUniversity%2520of%2520Sun

segmentedlist is now Parcelforce%2520Worldwide%3A25518%2CSaffery%2520Champness%3A25532%2CCroydon%2520Council%3A25679
companyparselist is now Bango%3A23533%2CIrwin%2520Mitchell%3A23668%2CNational%2520Express%2520Ltd%3A23686%2CLondon%2520Borough%2520of%2520Barnet%3A23701%2CSouthampton%2520City%2520Council%3A23856%2CWilliam%2520Grant%2520%2526%2520Sons%3A23883%2CTransics%3A23891%2COdfjell%2520Drilling%3A23938%2CInnocent%2520Drinks%3A24177%2CSouthwark%2520Council%3A24216%2CEducation%2520Development%2520Trust%3A24392%2CBoden%3A24463%2CWest%2520Lothian%2520Council%3A24599%2CConxion%3A24806%2CLincolnshire%2520County%2520Council%3A25135%2CBinckBank%3A25229%2CCity%2520Index%3A25251%2CCanterbury%2520Christ%2520Church%2520University%3A25262%2CStaffordshire%2520County%2520Council%3A25441%2CThe%2520Irish%2520Times%3A25458%2CSlough%2520Borough%2520Council%3A25472%2CNottinghamshire%2520County%2520Council%3A25503
288
segmentedlist is now The%2520Irish%2520Times%3A25458%2CSlough%2520Borough%2520Council

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=91.0.4472.114)
